**Описание данных:**
- `data_arc.csv` — данные об электродах;
- `data_bulk.csv` — данные о подаче сыпучих материалов (объём);
- `data_bulk_time.csv` *—* данные о подаче сыпучих материалов (время);
- `data_gas.csv` — данные о продувке сплава газом;
- `data_temp.csv` — результаты измерения температуры;
- `data_wire.csv` — данные о проволочных материалах (объём);
- `data_wire_time.csv` — данные о проволочных материалах (время).

Задача: построить модель, которая предскажет температуру стали.

**План исследования**

1) Исследовать исодные данные (загрузить, посмотреть значения в столбцах, изучить признаки); 
2) Предобработать данные (заполнить пропуски, исследовать выбросы, проанализировать аномалии);  
3) Выделить признаки, подобрать подходящую модель и метрики, которые будем проверять;
4) Обучить данные на обучающей выборке; 
5) Посмотреть метрику MSE на тестовой выборке.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
# Загрузим данные с температурой
data_temp = pd.read_csv('/datasets/final_steel/data_temp.csv')
display(data_temp)

,key,Время замера,Температура
0,1,2019-05-03 11:16:18,1571.0
1,1,2019-05-03 11:25:53,1604.0
2,1,2019-05-03 11:29:11,1618.0
3,1,2019-05-03 11:30:01,1601.0
4,1,2019-05-03 11:30:39,1613.0
...,...,...,...
15902,3241,2019-09-01 04:03:30,1586.0
15903,3241,2019-09-01 04:16:12,NaN
15904,3241,2019-09-01 04:22:39,NaN
15905,3241,2019-09-01 04:33:42,NaN


In [3]:
# Так как температура является для нас целевым значением, замена пропусков (например, на среднне значение),
#может помешать обучению, поэтому удалим все пропуски

data_temp = data_temp.dropna()
data_temp = data_temp.reset_index(drop=True)
print(data_temp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13006 entries, 0 to 13005
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   key           13006 non-null  int64  
 1   Время замера  13006 non-null  object 
 2   Температура   13006 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 305.0+ KB
None


In [4]:
# Заменим тип данных время замера на datetime
data_temp['Время замера'] = pd.to_datetime(data_temp['Время замера'], format='%Y-%m-%dT%H:%M:%S')
print(data_temp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13006 entries, 0 to 13005
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   key           13006 non-null  int64         
 1   Время замера  13006 non-null  datetime64[ns]
 2   Температура   13006 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 305.0 KB
None


In [5]:
# В качестве целевого признака возьмем только последнюю температуру по партии
data_temp_last = data_temp.drop_duplicates(subset='key', keep="last") 
data_temp_last = data_temp_last.reset_index(drop = True)
data_temp_last

,key,Время замера,Температура
0,1,2019-05-03 11:30:39,1613.0
1,2,2019-05-03 11:59:12,1602.0
2,3,2019-05-03 12:34:57,1599.0
3,4,2019-05-03 12:59:25,1625.0
4,5,2019-05-03 13:36:01,1602.0
...,...,...,...
3211,3237,2019-08-31 22:44:04,1569.0
3212,3238,2019-08-31 23:30:31,1584.0
3213,3239,2019-09-01 01:31:47,1598.0
3214,3240,2019-09-01 02:39:01,1617.0


## Подготовка фичей

В нашу таблицу с температурой data_temp_last будет добавлять признаки, которые влияют на значение тепературы. Важно, что для каждого шага выбираем только те фичи, которые были добавлены в смесь до последнего измерения.

### Сыпучие материалы

In [6]:
# Загрузим данные с материалами
data_bulk = pd.read_csv('/datasets/final_steel/data_bulk.csv')
display(data_bulk)

,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,NaN,NaN,NaN,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,150.0,154.0
1,2,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,149.0,154.0
2,3,NaN,NaN,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,152.0,153.0
3,4,NaN,NaN,NaN,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.0,NaN,153.0,154.0
4,5,NaN,NaN,NaN,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203.0,NaN,151.0,152.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,3237,NaN,NaN,170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252.0,NaN,130.0,206.0
3125,3238,NaN,NaN,126.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254.0,NaN,108.0,106.0
3126,3239,NaN,NaN,NaN,NaN,NaN,114.0,NaN,NaN,NaN,NaN,NaN,158.0,NaN,270.0,88.0
3127,3240,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.0,54.0


In [7]:
print(data_bulk.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   key      3129 non-null   int64  
 1   Bulk 1   252 non-null    float64
 2   Bulk 2   22 non-null     float64
 3   Bulk 3   1298 non-null   float64
 4   Bulk 4   1014 non-null   float64
 5   Bulk 5   77 non-null     float64
 6   Bulk 6   576 non-null    float64
 7   Bulk 7   25 non-null     float64
 8   Bulk 8   1 non-null      float64
 9   Bulk 9   19 non-null     float64
 10  Bulk 10  176 non-null    float64
 11  Bulk 11  177 non-null    float64
 12  Bulk 12  2450 non-null   float64
 13  Bulk 13  18 non-null     float64
 14  Bulk 14  2806 non-null   float64
 15  Bulk 15  2248 non-null   float64
dtypes: float64(15), int64(1)
memory usage: 391.2 KB
None


In [8]:
# Заменим пропуски на нулевые значения,
#так как пропуск свидетельствует о том, что ничего не добавлялось 
data_bulk = data_bulk.fillna(0)

In [9]:
#Чаще всего использовались следующие сыпучие материалы: 
#Bulk 3,4,12,14,15.
data_bulk = data_bulk[['key','Bulk 3','Bulk 4','Bulk 12','Bulk 14','Bulk 15']]
data_bulk

,key,Bulk 3,Bulk 4,Bulk 12,Bulk 14,Bulk 15
0,1,0.0,43.0,206.0,150.0,154.0
1,2,0.0,73.0,206.0,149.0,154.0
2,3,0.0,34.0,205.0,152.0,153.0
3,4,0.0,81.0,207.0,153.0,154.0
4,5,0.0,78.0,203.0,151.0,152.0
...,...,...,...,...,...,...
3124,3237,170.0,0.0,252.0,130.0,206.0
3125,3238,126.0,0.0,254.0,108.0,106.0
3126,3239,0.0,0.0,158.0,270.0,88.0
3127,3240,0.0,0.0,0.0,192.0,54.0


In [10]:
# Посмотрим на время добавления присыпок,
# так как нельзя использовать присыпки, 
# которые были сделаны после последнего замера температуры

data_bulk_time = pd.read_csv('/datasets/final_steel/data_bulk_time.csv')
data_bulk_time = data_bulk_time[['key','Bulk 3','Bulk 4','Bulk 12','Bulk 14','Bulk 15']]
display(data_bulk_time)

,key,Bulk 3,Bulk 4,Bulk 12,Bulk 14,Bulk 15
0,1,NaN,2019-05-03 11:21:30,2019-05-03 11:03:52,2019-05-03 11:03:52,2019-05-03 11:03:52
1,2,NaN,2019-05-03 11:46:38,2019-05-03 11:40:20,2019-05-03 11:40:20,2019-05-03 11:40:20
2,3,NaN,2019-05-03 12:31:06,2019-05-03 12:09:40,2019-05-03 12:09:40,2019-05-03 12:09:40
3,4,NaN,2019-05-03 12:48:43,2019-05-03 12:41:24,2019-05-03 12:41:24,2019-05-03 12:41:24
4,5,NaN,2019-05-03 13:18:50,2019-05-03 13:12:56,2019-05-03 13:12:56,2019-05-03 13:12:56
...,...,...,...,...,...,...
3124,3237,2019-08-31 22:51:28,NaN,2019-08-31 22:46:52,2019-08-31 22:46:52,2019-08-31 22:46:52
3125,3238,2019-08-31 23:39:11,NaN,2019-08-31 23:33:09,2019-08-31 23:33:09,2019-08-31 23:33:09
3126,3239,NaN,NaN,2019-09-01 01:39:41,2019-09-01 01:33:25,2019-09-01 01:33:25
3127,3240,NaN,NaN,NaN,2019-09-01 02:41:27,2019-09-01 02:41:27


In [11]:
# Добавим в таблицу с присыпками время последнего замера
data_bulk_time['Время_замера'] = data_temp_last['Время замера']
name_bulk = ('Bulk 3','Bulk 4','Bulk 12','Bulk 14','Bulk 15')

# Посмотрим, есть ли время добавления сыпучих материалов позже замеров
for i in name_bulk:
    print(data_bulk_time[data_bulk_time.Время_замера<data_bulk_time[i]])
    
# Таких значений нет

       key               Bulk 3 Bulk 4              Bulk 12  \
67      77  2019-05-06 02:28:27    NaN  2019-05-06 02:02:14   
68      78  2019-05-06 03:15:07    NaN  2019-05-06 02:55:17   
69      79  2019-05-06 04:19:48    NaN  2019-05-06 04:01:08   
70      82  2019-05-06 07:02:21    NaN                  NaN   
87      99  2019-05-06 19:44:28    NaN  2019-05-06 19:35:34   
...    ...                  ...    ...                  ...   
3118  3231  2019-08-31 19:34:18    NaN  2019-08-31 19:24:10   
3119  3232  2019-08-31 20:03:29    NaN  2019-08-31 19:58:36   
3123  3236  2019-08-31 22:20:30    NaN  2019-08-31 22:11:55   
3124  3237  2019-08-31 22:51:28    NaN  2019-08-31 22:46:52   
3125  3238  2019-08-31 23:39:11    NaN  2019-08-31 23:33:09   

                  Bulk 14              Bulk 15        Время_замера  
67    2019-05-06 02:02:14  2019-05-06 02:02:14 2019-05-05 21:42:24  
68    2019-05-06 02:55:17  2019-05-06 02:55:17 2019-05-05 22:26:54  
69    2019-05-06 04:01:08  2019-05-0

In [12]:
# Объединим данные по присыпкам с нашей таблицей с температурами
data_temp_last = data_temp_last.merge(data_bulk, how = 'inner')
data_temp_last

,key,Время замера,Температура,Bulk 3,Bulk 4,Bulk 12,Bulk 14,Bulk 15
0,1,2019-05-03 11:30:39,1613.0,0.0,43.0,206.0,150.0,154.0
1,2,2019-05-03 11:59:12,1602.0,0.0,73.0,206.0,149.0,154.0
2,3,2019-05-03 12:34:57,1599.0,0.0,34.0,205.0,152.0,153.0
3,4,2019-05-03 12:59:25,1625.0,0.0,81.0,207.0,153.0,154.0
4,5,2019-05-03 13:36:01,1602.0,0.0,78.0,203.0,151.0,152.0
...,...,...,...,...,...,...,...,...
3124,3237,2019-08-31 22:44:04,1569.0,170.0,0.0,252.0,130.0,206.0
3125,3238,2019-08-31 23:30:31,1584.0,126.0,0.0,254.0,108.0,106.0
3126,3239,2019-09-01 01:31:47,1598.0,0.0,0.0,158.0,270.0,88.0
3127,3240,2019-09-01 02:39:01,1617.0,0.0,0.0,0.0,192.0,54.0


### Электроды

In [13]:
data_arc = pd.read_csv('/datasets/final_steel/data_arc.csv')
display(data_arc)

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397
...,...,...,...,...,...
14871,3241,2019-09-01 03:58:58,2019-09-01 04:01:35,0.533670,0.354439
14872,3241,2019-09-01 04:05:04,2019-09-01 04:08:04,0.676604,0.523631
14873,3241,2019-09-01 04:16:41,2019-09-01 04:19:45,0.733899,0.475654
14874,3241,2019-09-01 04:31:51,2019-09-01 04:32:48,0.220694,0.145768


In [14]:
# Заменим тип данных время конца наргрева на datetime
data_arc['Конец нагрева дугой'] = pd.to_datetime(data_arc['Конец нагрева дугой'], format='%Y-%m-%dT%H:%M:%S')
print(data_arc.info())
data_arc

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   key                   14876 non-null  int64         
 1   Начало нагрева дугой  14876 non-null  object        
 2   Конец нагрева дугой   14876 non-null  datetime64[ns]
 3   Активная мощность     14876 non-null  float64       
 4   Реактивная мощность   14876 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 581.2+ KB
None


,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397
...,...,...,...,...,...
14871,3241,2019-09-01 03:58:58,2019-09-01 04:01:35,0.533670,0.354439
14872,3241,2019-09-01 04:05:04,2019-09-01 04:08:04,0.676604,0.523631
14873,3241,2019-09-01 04:16:41,2019-09-01 04:19:45,0.733899,0.475654
14874,3241,2019-09-01 04:31:51,2019-09-01 04:32:48,0.220694,0.145768


In [15]:
# Cуммируем по партиям
data_arc_new = data_arc.groupby(['key']).agg(sum)
display(data_arc_new)

,Активная мощность,Реактивная мощность
key,,
1,4.878147,3.183241
2,3.052598,1.998112
3,2.525882,1.599076
4,3.209250,2.060298
5,3.347173,2.252643
...,...,...
3237,3.932467,2.828459
3238,2.106529,1.681182
3239,4.666754,3.111617


In [16]:
# Находим время последнего нагрева
data_arc.drop_duplicates(subset = 'key', keep = "last") 
data_arc_new['time_power'] = data_arc['Конец нагрева дугой']

In [17]:
data_arc_new.reset_index()
display(data_arc_new)

,Активная мощность,Реактивная мощность,time_power
key,,,
1,4.878147,3.183241,2019-05-03 11:10:33
2,3.052598,1.998112,2019-05-03 11:14:36
3,2.525882,1.599076,2019-05-03 11:24:19
4,3.209250,2.060298,2019-05-03 11:28:37
5,3.347173,2.252643,2019-05-03 11:36:31
...,...,...,...
3237,3.932467,2.828459,2019-05-29 07:12:03
3238,2.106529,1.681182,2019-05-29 07:19:38
3239,4.666754,3.111617,2019-05-29 07:28:25


In [18]:
# Объединим таблицу
data_temp_last = data_temp_last.join(data_arc_new, on = 'key')
data_temp_last

,key,Время замера,Температура,Bulk 3,Bulk 4,Bulk 12,Bulk 14,Bulk 15,Активная мощность,Реактивная мощность,time_power
0,1,2019-05-03 11:30:39,1613.0,0.0,43.0,206.0,150.0,154.0,4.878147,3.183241,2019-05-03 11:10:33
1,2,2019-05-03 11:59:12,1602.0,0.0,73.0,206.0,149.0,154.0,3.052598,1.998112,2019-05-03 11:14:36
2,3,2019-05-03 12:34:57,1599.0,0.0,34.0,205.0,152.0,153.0,2.525882,1.599076,2019-05-03 11:24:19
3,4,2019-05-03 12:59:25,1625.0,0.0,81.0,207.0,153.0,154.0,3.209250,2.060298,2019-05-03 11:28:37
4,5,2019-05-03 13:36:01,1602.0,0.0,78.0,203.0,151.0,152.0,3.347173,2.252643,2019-05-03 11:36:31
...,...,...,...,...,...,...,...,...,...,...,...
3124,3237,2019-08-31 22:44:04,1569.0,170.0,0.0,252.0,130.0,206.0,3.932467,2.828459,2019-05-29 07:12:03
3125,3238,2019-08-31 23:30:31,1584.0,126.0,0.0,254.0,108.0,106.0,2.106529,1.681182,2019-05-29 07:19:38
3126,3239,2019-09-01 01:31:47,1598.0,0.0,0.0,158.0,270.0,88.0,4.666754,3.111617,2019-05-29 07:28:25
3127,3240,2019-09-01 02:39:01,1617.0,0.0,0.0,0.0,192.0,54.0,2.984083,2.483267,2019-05-29 07:57:30


In [19]:
# Если время нагрева больше времени снятия замера, то удаляем эту партию
data_temp_last = data_temp_last.drop(data_temp_last[data_temp_last['Время замера']<data_temp_last['time_power']].index,axis= 0)
# Удаляем столбец с time power
data_temp_last = data_temp_last.drop(['time_power'] , axis=1)

### Первичная температура

In [20]:
# добавим фактор первичная температура
data_temp_first = data_temp.drop_duplicates(subset='key', keep="first") 
data_temp_first = data_temp_first[['key','Температура']]
data_temp_first.rename(columns = {'Температура' : 'Первичная Т'}, inplace = True) 
data_temp_first

,key,Первичная Т
0,1,1571.0
5,2,1581.0
11,3,1596.0
16,4,1601.0
19,5,1576.0
...,...,...
13001,3237,1569.0
13002,3238,1584.0
13003,3239,1598.0
13004,3240,1617.0


In [21]:
data_temp_first  = data_temp_first.reset_index()
# объединим таблицы
data_temp_last['Первичная_температура'] = data_temp_first['Первичная Т']
display(data_temp_last)


,key,Время замера,Температура,Bulk 3,Bulk 4,Bulk 12,Bulk 14,Bulk 15,Активная мощность,Реактивная мощность,Первичная_температура
0,1,2019-05-03 11:30:39,1613.0,0.0,43.0,206.0,150.0,154.0,4.878147,3.183241,1571.0
1,2,2019-05-03 11:59:12,1602.0,0.0,73.0,206.0,149.0,154.0,3.052598,1.998112,1581.0
2,3,2019-05-03 12:34:57,1599.0,0.0,34.0,205.0,152.0,153.0,2.525882,1.599076,1596.0
3,4,2019-05-03 12:59:25,1625.0,0.0,81.0,207.0,153.0,154.0,3.209250,2.060298,1601.0
4,5,2019-05-03 13:36:01,1602.0,0.0,78.0,203.0,151.0,152.0,3.347173,2.252643,1576.0
...,...,...,...,...,...,...,...,...,...,...,...
3124,3237,2019-08-31 22:44:04,1569.0,170.0,0.0,252.0,130.0,206.0,3.932467,2.828459,1613.0
3125,3238,2019-08-31 23:30:31,1584.0,126.0,0.0,254.0,108.0,106.0,2.106529,1.681182,1629.0
3126,3239,2019-09-01 01:31:47,1598.0,0.0,0.0,158.0,270.0,88.0,4.666754,3.111617,1625.0
3127,3240,2019-09-01 02:39:01,1617.0,0.0,0.0,0.0,192.0,54.0,2.984083,2.483267,1596.0


In [22]:
# Удалим строки где первая температура равна последней
data_temp_last = data_temp_last.drop(data_temp_last[data_temp_last.Первичная_температура==data_temp_last.Температура].index,axis= 0)
data_temp_last
data_temp_last.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3088 entries, 0 to 3128
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   key                    3088 non-null   int64         
 1   Время замера           3088 non-null   datetime64[ns]
 2   Температура            3088 non-null   float64       
 3   Bulk 3                 3088 non-null   float64       
 4   Bulk 4                 3088 non-null   float64       
 5   Bulk 12                3088 non-null   float64       
 6   Bulk 14                3088 non-null   float64       
 7   Bulk 15                3088 non-null   float64       
 8   Активная мощность      3087 non-null   float64       
 9   Реактивная мощность    3087 non-null   float64       
 10  Первичная_температура  3088 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 289.5 KB


### Газ


In [23]:
# Загрузим данные с газом
data_gas = pd.read_csv('/datasets/final_steel/data_gas.csv')
display(data_gas)
data_gas.info()

,key,Газ 1
0,1,29.749986
1,2,12.555561
2,3,28.554793
3,4,18.841219
4,5,5.413692
...,...,...
3234,3237,5.543905
3235,3238,6.745669
3236,3239,16.023518
3237,3240,11.863103


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3239 entries, 0 to 3238
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   key     3239 non-null   int64  
 1   Газ 1   3239 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 50.7 KB


In [24]:
# Добавим как фактор в нашу таблицу
data_temp_last = data_temp_last.merge(data_gas, how = 'inner')

### Проволока

In [25]:
data_wire = pd.read_csv('/datasets/final_steel/data_wire.csv')
display(data_wire)

,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,60.059998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,96.052315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,91.160157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,89.063515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,89.238236,9.11456,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3076,3237,38.088959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3077,3238,56.128799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3078,3239,143.357761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3079,3240,34.070400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Заполним пропуски нулями (анологично присыпкам)
data_wire.fillna(0)
print(data_wire.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   key     3081 non-null   int64  
 1   Wire 1  3055 non-null   float64
 2   Wire 2  1079 non-null   float64
 3   Wire 3  63 non-null     float64
 4   Wire 4  14 non-null     float64
 5   Wire 5  1 non-null      float64
 6   Wire 6  73 non-null     float64
 7   Wire 7  11 non-null     float64
 8   Wire 8  19 non-null     float64
 9   Wire 9  29 non-null     float64
dtypes: float64(9), int64(1)
memory usage: 240.8 KB
None


In [27]:
data_wire_time = pd.read_csv('/datasets/final_steel/data_wire_time.csv')
display(data_wire_time)

,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,2019-05-03 11:11:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2019-05-03 11:46:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2019-05-03 12:13:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2019-05-03 12:48:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2019-05-03 13:18:15,2019-05-03 13:32:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3076,3237,2019-08-31 22:50:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3077,3238,2019-08-31 23:38:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3078,3239,2019-09-01 01:50:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3079,3240,2019-09-01 03:12:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Удалим где по времени позже для Wire 1
# Объединим количество проволоки с временм подачи и временем последнего измерения температуры
wires = ('Wire 1','Wire 2')

for i in wires:
    data_wire_time_1 = data_wire_time[[i, 'key']]
    data_wire_time_1.rename(columns = {i : 'Wire_1_time'},inplace = True)
    data_wire_1 = data_wire[[i,'key']]

    data_wire_time_1 = data_wire_time_1.merge(data_wire_1, on='key')

    data_temp_last = data_temp_last.merge(data_wire_time_1,on='key')
    data_temp_last

    data_temp_last = data_temp_last.drop(data_temp_last[data_temp_last['Время замера']<data_temp_last['Wire_1_time']].index,axis= 0)

    data_temp_last  = data_temp_last.drop(['Wire_1_time'] , axis=1)


/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [29]:
data_temp_last.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2318 entries, 0 to 2332
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   key                    2318 non-null   int64         
 1   Время замера           2318 non-null   datetime64[ns]
 2   Температура            2318 non-null   float64       
 3   Bulk 3                 2318 non-null   float64       
 4   Bulk 4                 2318 non-null   float64       
 5   Bulk 12                2318 non-null   float64       
 6   Bulk 14                2318 non-null   float64       
 7   Bulk 15                2318 non-null   float64       
 8   Активная мощность      2317 non-null   float64       
 9   Реактивная мощность    2317 non-null   float64       
 10  Первичная_температура  2318 non-null   float64       
 11  Газ 1                  2318 non-null   float64       
 12  Wire 1                 2293 non-null   float64       
 13  Wir

In [30]:
data_temp_last = data_temp_last.fillna(0)
data_temp_last

,key,Время замера,Температура,Bulk 3,Bulk 4,Bulk 12,Bulk 14,Bulk 15,Активная мощность,Реактивная мощность,Первичная_температура,Газ 1,Wire 1,Wire 2
0,1,2019-05-03 11:30:39,1613.0,0.0,43.0,206.0,150.0,154.0,4.878147,3.183241,1571.0,29.749986,60.059998,0.000000
1,2,2019-05-03 11:59:12,1602.0,0.0,73.0,206.0,149.0,154.0,3.052598,1.998112,1581.0,12.555561,96.052315,0.000000
2,3,2019-05-03 12:34:57,1599.0,0.0,34.0,205.0,152.0,153.0,2.525882,1.599076,1596.0,28.554793,91.160157,0.000000
3,4,2019-05-03 12:59:25,1625.0,0.0,81.0,207.0,153.0,154.0,3.209250,2.060298,1601.0,18.841219,89.063515,0.000000
4,5,2019-05-03 13:36:01,1602.0,0.0,78.0,203.0,151.0,152.0,3.347173,2.252643,1576.0,5.413692,89.238236,9.114560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,3164,2019-08-29 11:56:59,1595.0,286.0,0.0,254.0,154.0,158.0,3.716570,2.521006,1624.0,7.620290,109.243675,0.000000
2329,3186,2019-08-30 09:45:52,1591.0,140.0,0.0,258.0,150.0,106.0,2.456302,1.857677,1616.0,5.699542,130.166403,0.000000
2330,3197,2019-08-30 18:56:00,1580.0,154.0,0.0,308.0,148.0,206.0,1.788605,1.335075,1595.0,1.837179,30.008158,90.178398
2331,3215,2019-08-31 07:21:28,1547.0,0.0,0.0,0.0,0.0,0.0,0.883352,0.808736,1591.0,3.129189,4.062240,0.000000


## Обучение модели

In [31]:
#Обучение
target = data_temp_last['Температура']
features = data_temp_last[['Первичная_температура','Wire 1','Wire 2','Газ 1','Активная мощность','Bulk 3','Bulk 4','Bulk 12','Bulk 14','Bulk 15']]

In [33]:
features

,Первичная_температура,Wire 1,Wire 2,Газ 1,Активная мощность,Bulk 3,Bulk 4,Bulk 12,Bulk 14,Bulk 15
0,1571.0,60.059998,0.000000,29.749986,4.878147,0.0,43.0,206.0,150.0,154.0
1,1581.0,96.052315,0.000000,12.555561,3.052598,0.0,73.0,206.0,149.0,154.0
2,1596.0,91.160157,0.000000,28.554793,2.525882,0.0,34.0,205.0,152.0,153.0
3,1601.0,89.063515,0.000000,18.841219,3.209250,0.0,81.0,207.0,153.0,154.0
4,1576.0,89.238236,9.114560,5.413692,3.347173,0.0,78.0,203.0,151.0,152.0
...,...,...,...,...,...,...,...,...,...,...
2328,1624.0,109.243675,0.000000,7.620290,3.716570,286.0,0.0,254.0,154.0,158.0
2329,1616.0,130.166403,0.000000,5.699542,2.456302,140.0,0.0,258.0,150.0,106.0
2330,1595.0,30.008158,90.178398,1.837179,1.788605,154.0,0.0,308.0,148.0,206.0
2331,1591.0,4.062240,0.000000,3.129189,0.883352,0.0,0.0,0.0,0.0,0.0


In [32]:
features_train, features_test, target_train, target_test = train_test_split(
   features, target, test_size=0.25, random_state=12345) # разбиваем на обучающую и тестовую выборку

model = RandomForestRegressor(n_estimators=110, max_depth=8, random_state=12345)
model.fit(features_train, target_train) 
predictions_test = model.predict(features_test) 

print("MAE на тестовой выборке: ", mean_absolute_error(target_test, predictions_test ))


MAE на тестовой выборке:  6.812207320204699


**Результат:** В результе первичного анализа в качестве признаков были выбраны: Первичная_температура, Wire 1,Wire 2, Газ 1, Активная мощность, Bulk 3, Bulk 4, Bulk 12, Bulk 14, Bulk 15. С помошью модели RandomForestRegressor была получена MAE равная 6,8.